# Combine into single dataset 
    A) Satelite data from Google Earth engine 
    B) desinventar events 
    C) set of drought events found by searching for news articles 

Every row in the dataframe will correspond to a particular month/year + region. 
For every drought event, we add a "TRUE" to the collumn corresponding to the reported date. 
Also have the option to extend this event by, for example, ±1 month

In [34]:
%matplotlib inline 
import pandas as pd 
import matplotlib.pylab as plt 
import numpy as np 
from datetime import date

In [35]:
def replace_underscos(cols,rep='-'):
    cols = list(cols)
    new_cols = list(map(lambda x: x.replace('_',rep), cols))
    col_dict = dict(zip(cols, new_cols))
    return col_dict

# Satelite data from Google Earth engine

for general description:
https://developers.google.com/earth-engine/datasets

The datasets/ collumn names we use for now: 
* NDVI: 'Normalized Difference Vegitation Index" (scale by 0.0001)
* EVI: 'Enhanced Vegitation Index' (scale by 0.0001)
* precipitation: rainfall measured in mm/hrs (summed over a month to get mm/month)
* hourlyPrecipRate: (also?) rainfall measured in mm/hrs (summed over month to get mm/month)
* LST_Day_1km: Land surface temperature during daytime in  50 Kelvins (multiply number by 0.02 to get Kelvins)
* LST_Night_1km: Land surface temperature during nighttime in  50 Kelvins (multiply number by 0.02 to get Kelvins)
* Evap_tavg: Evapotranspiration measured in kg m^{-2} s^{-1}  ("flux per second") 
* Rainf_f_tavg: Total precipitation rate measured in kg m^{-2} s^{-1}  ("flux per second") 
* SoilMoi00_10cm_tavg: Soil moisture (0 - 10 cm underground)    in m^3 m-3
* SoilMoi10_40cm_tavg: Soil moisture (10 - 40 cm underground)   in m^3 m-3
* SoilMoi40_100cm_tavg: Soil moisture (40 - 100 cm underground) in m^3 m-3
* SoilMoi100_200cm_tavg: Soil moisture (100 - 200 cm underground) in m^3 m-3
* SoilTemp00_10cm_tavg: Soil temperature (0 - 10 cm underground) in K 
* SoilTemp0_40cm_tavg: Soil temperature (10 - 40 cm underground) in K 
* SoilTemp40_100cm_tavg: Soil temperature (40 - 100 cm underground) in K 
* SoilTemp100_200cm_tavg:Soil temperature (100 - 200 cm underground) in K 
* Tair_f_tavg: Near surface air temperature in Kelvin
* Wind_f_tavg: Near surface wind speed in m/s 

using some built-in R-code we further determined 
* SPEI: A measure of precipitation minus evapotranspiration. We have these averaged over the last 1, 2, 3, ...,12 months


we data have two countries: Uganda and Kenya (admin level 1 refers to a district, Red Cross terminology) 

In [36]:
UG_satelite = pd.read_csv('datasets/UG_merged_adm1_with_spei.csv')
KE_satelite = pd.read_csv('datasets/KE_merged_adm1_with_spei.csv')

### prepare the date column 
We are working with monthly averages, so we just use the 1st of the month as a placeholder. 
Although event data is available for particular publication days, I here do not use that information (see below when combining), as this is not an accurate indicator of the actual drought event anyways.  

The following is simply in order to visualize time series later on. 

In [37]:
def get_date(row):
    try:
        year = row['Year']
        month= row['Month']
        day = row['Day']
    except:
        year = row['year']
        month= row['month']
        day = row['day']
    
    try:
        date_of_event = date(year, month, day)
    except:
        # --- if we do not have the day, that is OK for drought, 
        # --- just put it on the 1st of the month --- 
        if (year!=0 and month!=0):
            date_of_event = date(year,month,1)
        else:
            date_of_event = np.nan
    return date_of_event

UG_satelite['Day'] = 1   # As we are working with 
UG_satelite['Date'] = UG_satelite.apply(get_date, axis=1)

KE_satelite['Day'] = 1   # As we are working with 
KE_satelite['Date'] = KE_satelite.apply(get_date, axis=1)

### rename collumns to have (slightly) more intuitive names 

In [38]:
columns_to_keep_satelite = { 'ADM0_EN': 'Country',
                    'ADM1_EN': 'District',
                    'Year':'year',
                    'Month':'month',
                    'Day':'day',
                    'Date':'date',
                    'NDVI_mean':'NDVI',
                    'EVI_mean': 'EVI',
                    'precipitation_sum':'precipitation_per_hour_v1',
                    'hourlyPrecipRate_sum':'precipitation_per_hour_v2',
                   'LST_Day_1km_mean':'surface_temperature_daytime',
                   'LST_Night_1km_mean':'surface_temperature_nighttime',
                    'Evap_tavg_mean': 'evapotranspiration',
                    'Rainf_f_tavg_mean':'rainfall',
                    'SoilMoi00_10cm_tavg_mean':'SoilMoisture00_10cm',
                    'SoilMoi10_40cm_tavg_mean':'SoilMoisture10_40cm',
                    'SoilMoi40_100cm_tavg_mean':'SoilMoisture40_100cm',
                    'SoilMoi100_200cm_tavg_mean':'SoilMoisture100_200cm',
                    'SoilTemp00_10cm_tavg_mean':'SoilTemperature00_10cm',
                    'SoilTemp10_40cm_tavg_mean':'SoilTemperature10_40cm',
                    'SoilTemp40_100cm_tavg_mean':'SoilTemperature40_100cm',
                    'SoilTemp100_200cm_tavg_mean':'SoilTemperature100_200cm',
                    'Tair_f_tavg_mean': 'air_temperature',
                    'Wind_f_tavg_mean': 'wind_speed',
                    'spei_1':'SPEI_1month',
                    'spei_2':'SPEI_2month',
                    'spei_3':'SPEI_3month',
                    'spei_4':'SPEI_4month',
                    'spei_5':'SPEI_5month',
                    'spei_6':'SPEI_6month',
                    'spei_7':'SPEI_7month',
                    'spei_8':'SPEI_8month',
                    'spei_9':'SPEI_9month',
                    'spei_10':'SPEI_10month',
                    'spei_11':'SPEI_11month',
                    'spei_12':'SPEI_12month'
                   }




UG = UG_satelite[columns_to_keep_satelite.keys()].copy()
UG.rename(columns=columns_to_keep_satelite, inplace=True)

KE = KE_satelite[columns_to_keep_satelite.keys()].copy()
KE.rename(columns=columns_to_keep_satelite, inplace=True)


### Combine the countries into a single dataframe/file

In [39]:
DroughtData = pd.concat([UG,KE])
DroughtData.reset_index(drop=True,inplace=True)
DroughtData.head()

,Country,District,year,month,day,date,NDVI,EVI,precipitation_per_hour_v1,precipitation_per_hour_v2,...,SPEI_3month,SPEI_4month,SPEI_5month,SPEI_6month,SPEI_7month,SPEI_8month,SPEI_9month,SPEI_10month,SPEI_11month,SPEI_12month
0,Uganda,ABIM,2000,3,1,2000-03-01,2707.118608,1371.556048,0.031873,5.285528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Uganda,ABIM,2000,4,1,2000-04-01,3684.379945,1846.158551,0.172345,84.424662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Uganda,ABIM,2000,5,1,2000-05-01,6247.636389,3653.801670,0.185952,100.613865,...,0.075238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Uganda,ABIM,2000,6,1,2000-06-01,6698.680552,4168.914213,0.205662,91.973897,...,0.917779,0.711107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uganda,ABIM,2000,7,1,2000-07-01,7177.464449,4584.023762,0.157562,67.037938,...,1.070958,0.836407,0.635385,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### transform to proper units 

In [40]:
DroughtData['NDVI'] *= 0.0001
DroughtData['EVI'] *= 0.0001
DroughtData['surface_temperature_daytime'] *= 0.02
DroughtData['surface_temperature_nighttime'] *= 0.02

# Droughts reported in the news 

In [41]:
import pandas as pd
droughts_from_news = pd.read_csv('datasets/news_droughts_Uganda_districts.csv',index_col=0)
droughts_from_news.reset_index(inplace=True,drop=True)

In [42]:
droughts_from_news.head()

,date,subregion,district,county,subcounty,sentence,article_title,article_url
0,2017-09-02,NaN,AMUDAT,POKOT,NaN,"These include Baringo, Garissa, Isiolo, Marsab...","Kenya experiencing one of the worst drought, d...",https://capitalradio.co.ug/kenya-experiencing-...
1,2017-07-09,NaN,"NTUNGAMO, BUSHENYI, MARACHA, IGANGA, NEBBI, NA...","ISINGIRO, MARACHA","NAMUTUMBA, NTUNGAMO, NEBBI, KIRYANDONGO","They include Arua, Maracha, Nebbi, Yumbe, Igan...","Ugandans are faced with food insecurity, offic...",https://capitalradio.co.ug/ugandans-faced-food...
2,2017-04-24,NaN,GULU,NaN,NaN,Persistent drought dries up Dam in Gulu.,Persistent drought dries up Dam in Gulu,https://capitalradio.co.ug/persistent-drought-...
3,2017-04-24,NaN,GULU,NaN,NaN,Persistent drought dries up Dam in Gulu,Persistent drought dries up Dam in Gulu,https://capitalradio.co.ug/persistent-drought-...
4,2017-04-24,NaN,GULU,NaN,NaN,Two large dams supplying water to Gulu town ha...,Persistent drought dries up Dam in Gulu,https://capitalradio.co.ug/persistent-drought-...


### complete dataframe (missing values)
Some of the events do not have a reported district, but do still have lower/higher level regions specified. Sometimes the quote ('sentence' collumn) has a list of districts. 

For now, I ignored the events without explicit mentioning of the district. Should probably complete dataframe later. 


### prepare merge with satelite data
Make collumns with the month + year (+ day). When comparing to satelite data, we can then check for the district + year + month.  


In [43]:
def get_day_month_year(date_str):
    date = date_str.split('-')
    year = date[0]
    month = date[1]
    day = date[2]
    return day, month, year



days = []
months = []
years  = []

for i in range(len(droughts_from_news)):
    day, month, year = get_day_month_year(droughts_from_news['date'][i])
    days.append(day)
    
    if month[0] == '0':
        months.append(month[1])
    else: 
        months.append(month)
    years.append(year)

droughts_from_news['year'] = years
droughts_from_news['month'] = months
droughts_from_news['day'] = days 

### loop over all reported events (with a district name, see above) and manually merge with satelite data
As a single district can have multiple events reported and a single event can apply to several districts, I decided to make a loop to merge it with satelite data. 



In [44]:
droughts_from_news = droughts_from_news[droughts_from_news['district'].notnull()]


DroughtData['drought_reported'] = False
DroughtData['drought_news_article'] = False

# --- loop over events --- 
for i in droughts_from_news.index:
    # --- extract the districts belonging to the reported event ---- 
    districts_effected = droughts_from_news['district'][i].split(',')
    
    # --- filter the satelite data by the reported date + district ---- 
    for dist in districts_effected:
        subset = DroughtData[(DroughtData['District']==dist )& 
                             (DroughtData['month'] == int(droughts_from_news.loc[i]['month']))&
                             (DroughtData['year'] == int(droughts_from_news.loc[i]['year']))]
        
        
        # --- record that there has been an event ---- 
        for j in subset.index:
            DroughtData.loc[j,'drought_reported'] = True
        # --- record the event comes from news articles --- 
            DroughtData.loc[j,'drought_news_article'] = True

        
print(len(DroughtData[DroughtData['drought_reported']]), len(DroughtData[DroughtData['drought_news_article']]))

81 81


# Desinventar data 

similar as above. Just keep track of which events where in desinventar

In [45]:
UG_desinventar = pd.read_csv('datasets/DI_export_uga.csv')
UG_droughts = UG_desinventar[UG_desinventar['event']=="DROUGHT"].copy()

KE_desinventar = pd.read_csv('datasets/DI_export_ken.csv')
KE_droughts = KE_desinventar[KE_desinventar['event']=="DROUGHT"].copy()

# --- actual day of the event, only use if we have year + month ---  
UG_droughts['date_of_event'] = KE_droughts.apply(get_date,axis=1)
KE_droughts['date_of_event'] = KE_droughts.apply(get_date,axis=1)


# --- overwrite the day to be the 1st of the month in order to do the merging
UG_droughts['day'] = 1 
UG_droughts['date'] = UG_droughts.apply(get_date,axis=1)

KE_droughts['day'] = 1 
KE_droughts['date'] = UG_droughts.apply(get_date,axis=1)

# --- for merge --- 
UG_droughts['Country'] = 'Uganda'
KE_droughts['Country'] = 'Kenya'

columns_to_keep_desinventar = {'Country':'Country',
                              'admin_level_0_name':'District',
                              'year':'year',
                              'month':'month',
                              'day':'day',
                              'date':'date',
                              'date_of_event':'date_of_event',
                              'latitude':'latitude',
                              'longitude':'longitude'}

UG_droughts_RK = UG_droughts[columns_to_keep_desinventar.keys()].copy()
UG_droughts_RK.rename(columns=columns_to_keep_desinventar, inplace=True);

KE_droughts_RK = KE_droughts[columns_to_keep_desinventar.keys()].copy()
KE_droughts_RK.rename(columns=columns_to_keep_desinventar, inplace=True);

### Combine the countries into a single dataframe/file

In [46]:
desinventar = pd.concat([UG_droughts_RK,KE_droughts_RK])
desinventar.reset_index(inplace=True, drop=True)

## no reported districts 
same as done for the news articles. Most probably at another level than 'admin level 1' the region might still be indicated in desinventar. 

In [47]:
desinventar = desinventar[desinventar['District'].notnull()]
print(len(desinventar))

1229


## properly combine this with the satelite data 
similar as done above for events from news articles 

In [48]:
# DroughtData['drought_reported'] = False
DroughtData['drought_desinventar'] = False
print(len(DroughtData[DroughtData['drought_reported']]), len(DroughtData[DroughtData['drought_news_article']]))

# --- loop over events --- 
for i in desinventar.index:
    # --- extract the districts belonging to the reported event ---- 
    districts_effected = desinventar['District'][i].upper()
    
    # --- filter the satelite data by the reported date + district ---- 
    subset = DroughtData[(DroughtData['District'].str.upper()==districts_effected )& 
                             (DroughtData['month'] == int(desinventar.loc[i]['month']))&
                             (DroughtData['year'] == int(desinventar.loc[i]['year']))]

    # --- record that there has been an event ---- 
    for j in subset.index:
        DroughtData.at[j, 'drought_reported'] = True
    # --- record the event comes from news articles --- 
        DroughtData.at[j, 'drought_desinventar'] = True

print(len(DroughtData[DroughtData['drought_reported']]),
      len(DroughtData[DroughtData['drought_news_article']]),
      len(DroughtData[DroughtData['drought_desinventar']]))

81 81
361 81 282


In [49]:
cols = DroughtData.columns
col_dict = replace_underscos(cols,rep=' ')
DroughtData.rename(columns=col_dict,inplace=True)

## save to a csv file

In [50]:
DroughtData.to_csv('datasets/Droughts_satelite_and_events.csv', index=False)

In [51]:
DroughtData.head()

,Country,District,year,month,day,date,NDVI,EVI,precipitation per hour v1,precipitation per hour v2,...,SPEI 6month,SPEI 7month,SPEI 8month,SPEI 9month,SPEI 10month,SPEI 11month,SPEI 12month,drought reported,drought news article,drought desinventar
0,Uganda,ABIM,2000,3,1,2000-03-01,0.270712,0.137156,0.031873,5.285528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1,Uganda,ABIM,2000,4,1,2000-04-01,0.368438,0.184616,0.172345,84.424662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
2,Uganda,ABIM,2000,5,1,2000-05-01,0.624764,0.365380,0.185952,100.613865,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
3,Uganda,ABIM,2000,6,1,2000-06-01,0.669868,0.416891,0.205662,91.973897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
4,Uganda,ABIM,2000,7,1,2000-07-01,0.717746,0.458402,0.157562,67.037938,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
